<a href="https://colab.research.google.com/github/ShinyRyo/matsuoken/blob/master/ra_men.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://qiita.com/toshiyuki_tsutsui/items/f143946944a428ed105b

In [0]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

class Tabelog:
    """
    食べログスクレイピングクラス
    test_mode=Trueで動作させると、最初のページの３店舗のデータのみを取得できる
    """
    def __init__(self, base_url, test_mode=False, p_ward='東京都内', begin_page=1, end_page=3):

        # 変数宣言
        self.store_id = ''
        self.store_id_num = 0
        self.store_name = ''
        self.score = 0
        self.ward = p_ward
        self.review_cnt = 0
        self.review = ''
        self.columns = ['store_id', 'store_name', 'score', 'ward', 'review_cnt', 'review']
        self.df = pd.DataFrame(columns=self.columns)
        self.__regexcomp = re.compile(r'\n|\s') # \nは改行、\sは空白

        page_num = begin_page # 店舗一覧ページ番号

        if test_mode:
            list_url = base_url + str(page_num) +  '/?Srt=D&SrtT=rt&sort_mode=1' #食べログの点数ランキングでソートする際に必要な処理
            self.scrape_list(list_url, mode=test_mode)
        else:
            while True:
                list_url = base_url + str(page_num) +  '/?Srt=D&SrtT=rt&sort_mode=1' #食べログの点数ランキングでソートする際に必要な処理
                if self.scrape_list(list_url, mode=test_mode) != True:
                    break

                # INパラメータまでのページ数データを取得する
                if page_num >= end_page:
                    break
                page_num += 1
        return

    def scrape_list(self, list_url, mode):
        """
        店舗一覧ページのパーシング
        """
        r = requests.get(list_url)
        if r.status_code != requests.codes.ok:
            return False

        soup = BeautifulSoup(r.content, 'html.parser')
        soup_a_list = soup.find_all('a', class_='list-rst__rst-name-target') # 店名一覧

        if len(soup_a_list) == 0:
            return False

        if mode:
            for soup_a in soup_a_list[:2]:
                item_url = soup_a.get('href') # 店の個別ページURLを取得
                self.store_id_num += 1
                self.scrape_item(item_url, mode)
        else:
            for soup_a in soup_a_list:
                item_url = soup_a.get('href') # 店の個別ページURLを取得
                self.store_id_num += 1
                self.scrape_item(item_url, mode)

        return True

    def scrape_item(self, item_url, mode):
        """
        個別店舗情報ページのパーシング
        """
        start = time.time()

        r = requests.get(item_url)
        if r.status_code != requests.codes.ok:
            print(f'error:not found{ item_url }')
            return

        soup = BeautifulSoup(r.content, 'html.parser')

        # 店舗名称取得
        # <h2 class="display-name">
        #     <span>
        #         麺匠　竹虎 新宿店
        #     </span>
        # </h2>
        store_name_tag = soup.find('h2', class_='display-name')
        store_name = store_name_tag.span.string
        print('{}→店名：{}'.format(self.store_id_num, store_name.strip()), end='')
        self.store_name = store_name.strip()

        # ラーメン屋、つけ麺屋以外の店舗は除外
        store_head = soup.find('div', class_='rdheader-subinfo') # 店舗情報のヘッダー枠データ取得
        store_head_list = store_head.find_all('dl')
        store_head_list = store_head_list[1].find_all('span')
        #print('ターゲット：', store_head_list[0].text)

        if store_head_list[0].text not in {'ラーメン', 'つけ麺'}:
            print('ラーメンorつけ麺のお店ではないので処理対象外')
            self.store_id_num -= 1
            return

        # 評価点数取得
        #<b class="c-rating__val rdheader-rating__score-val" rel="v:rating">
        #    <span class="rdheader-rating__score-val-dtl">3.58</span>
        #</b>
        rating_score_tag = soup.find('b', class_='c-rating__val')
        rating_score = rating_score_tag.span.string
        print('  評価点数：{}点'.format(rating_score), end='')
        self.score = rating_score

        # 評価点数が存在しない店舗は除外
        if rating_score == '-':
            print('  評価がないため処理対象外')
            self.store_id_num -= 1
            return
       # 評価が3.5未満店舗は除外
        if float(rating_score) < 3.5:
            print('  食べログ評価が3.5未満のため処理対象外')
            self.store_id_num -= 1
            return

        # レビュー一覧URL取得
        #<a class="mainnavi" href="https://tabelog.com/tokyo/A1304/A130401/13143442/dtlrvwlst/"><span>口コミ</span><span class="rstdtl-navi__total-count"><em>60</em></span></a>
        review_tag_id = soup.find('li', id="rdnavi-review")
        review_tag = review_tag_id.a.get('href')

        # レビュー件数取得
        print('  レビュー件数：{}'.format(review_tag_id.find('span', class_='rstdtl-navi__total-count').em.string), end='')
        self.review_cnt = review_tag_id.find('span', class_='rstdtl-navi__total-count').em.string

        # レビュー一覧ページ番号
        page_num = 3 #1ページ*20 = 20レビュー 。この数字を変えて取得するレビュー数を調整。

        # レビュー一覧ページから個別レビューページを読み込み、パーシング
        # 店舗の全レビューを取得すると、食べログの評価ごとにデータ件数の濃淡が発生してしまうため、
        # 取得するレビュー数は１ページ分としている（件数としては１ページ*20=２0レビュー）
        while True:
            review_url = review_tag + 'COND-0/smp1/?lc=0&rvw_part=all&PG=' + str(page_num)
            #print('\t口コミ一覧リンク：{}'.format(review_url))
            print(' . ' , end='') #LOG
            if self.scrape_review(review_url) != True:
                break
            if page_num >= 1:
                break
            page_num += 1

        process_time = time.time() - start
        print('  取得時間：{}'.format(process_time))

        return

    def scrape_review(self, review_url):
        """
        レビュー一覧ページのパーシング
        """
        r = requests.get(review_url)
        if r.status_code != requests.codes.ok:
            print(f'error:not found{ review_url }')
            return False

        # 各個人の口コミページ詳細へのリンクを取得する
        #<div class="rvw-item js-rvw-item-clickable-area" data-detail-url="/tokyo/A1304/A130401/13141542/dtlrvwlst/B408082636/?use_type=0&amp;smp=1">
        #</div>
        soup = BeautifulSoup(r.content, 'html.parser')
        review_url_list = soup.find_all('div', class_='rvw-item') # 口コミ詳細ページURL一覧

        if len(review_url_list) == 0:
            return False

        for url in review_url_list:
            review_detail_url = 'https://tabelog.com' + url.get('data-detail-url')
            #print('\t口コミURL：', review_detail_url)

            # 口コミのテキストを取得
            self.get_review_text(review_detail_url)

        return True

    def get_review_text(self, review_detail_url):
        """
        口コミ詳細ページをパーシング
        """
        r = requests.get(review_detail_url)
        if r.status_code != requests.codes.ok:
            print(f'error:not found{ review_detail_url }')
            return

        # ２回以上来訪してコメントしているユーザは最新の1件のみを採用
        #<div class="rvw-item__rvw-comment" property="v:description">
        #  <p>
        #    <br>すごい煮干しラーメン凪 新宿ゴールデン街本館<br>スーパーゴールデン1600円（20食限定）を喰らう<br>大盛り無料です<br>スーパーゴールデンは、新宿ゴールデン街にちなんで、ココ本店だけの特別メニューだそうです<br>相方と歌舞伎町のtohoシネマズの映画館でドラゴンボール超ブロリー を観てきた<br>ブロリー 強すぎるね(^^)面白かったです<br>凪の煮干しラーメンも激ウマ<br>いったん麺ちゅるちゅる感に、レアチャーと大トロチャーシューのトロけ具合もうめえ<br>煮干しスープもさすが！と言うほど完成度が高い<br>さすが食べログラーメン百名店<br>と言うか<br>2日連チャンで、近場の食べログラーメン百名店のうちの2店舗、昨日の中華そば葉山さんと今日の凪<br>静岡では考えられん笑笑<br>ごちそうさまでした
        #  </p>
        #</div>
        soup = BeautifulSoup(r.content, 'html.parser')
        review = soup.find_all('div', class_='rvw-item__rvw-comment')#reviewが含まれているタグの中身をすべて取得
        if len(review) == 0:
            review = ''
        else:
            review = review[0].p.text.strip() # strip()は改行コードを除外する関数

        #print('\t\t口コミテキスト：', review)
        self.review = review

        # データフレームの生成
        self.make_df()
        return

    def make_df(self):
        self.store_id = str(self.store_id_num).zfill(8) #0パディング
        se = pd.Series([self.store_id, self.store_name, self.score, self.ward, self.review_cnt, self.review], self.columns) # 行を作成
        self.df = self.df.append(se, self.columns) # データフレームに行を追加
        return

In [0]:
tokyo_ramen_review = Tabelog(base_url="https://tabelog.com/tokyo/rstLst/ramen/",test_mode=False, p_ward='東京都内')
#CSV保存

1→店名：らぁ麺 すぎ本  評価点数：4.09点  レビュー件数：413 .   取得時間：46.68645691871643
2→店名：手打式超多加水麺 ののくら  評価点数：4.02点  レビュー件数：300 .   取得時間：42.417338132858276
3→店名：中華そば しば田  評価点数：3.99点  レビュー件数：668 .   取得時間：47.59699463844299
4→店名：麺尊 RAGE  評価点数：3.98点  レビュー件数：526 .   取得時間：43.78718543052673
5→店名：麺庵ちとせ  評価点数：3.97点  レビュー件数：328 .   取得時間：42.17769503593445
6→店名：メンドコロ キナリ  評価点数：3.97点  レビュー件数：183 .   取得時間：40.15544867515564
7→店名：中華そば 多賀野  評価点数：3.96点  レビュー件数：736 .   取得時間：46.268303632736206
8→店名：らーめん改  評価点数：3.96点  レビュー件数：417 .   取得時間：41.27645540237427
9→店名：麺屋 さくら井  評価点数：3.95点  レビュー件数：324 .   取得時間：41.86659789085388
10→店名：ほっこり中華そば もつけ  評価点数：3.95点  レビュー件数：280 .   取得時間：40.353333473205566
11→店名：中華ソバ 伊吹  評価点数：3.94点  レビュー件数：431 .   取得時間：43.982011795043945
12→店名：中華蕎麦にし乃  評価点数：3.94点  レビュー件数：379 .   取得時間：39.343610525131226
13→店名：RAMEN　にじゅうぶんのいち  評価点数：3.94点  レビュー件数：201 .   取得時間：44.25706672668457
14→店名：饗 くろ喜  評価点数：3.93点  レビュー件数：893 .   取得時間：47.63735032081604
15→店名：SOBA HOUSE 金色不如帰 新宿御苑本店  評価点数：3.93点  レビュー件数：457 .   取得時間：46.03836441040

In [0]:
tokyo_ramen_review.df.to_csv("tokyo_ramen_review.csv")

In [0]:
tokyo_ramen_review = pd.read_csv('tokyo_ramen_review.csv')
tokyo_ramen_review

,Unnamed: 0,store_id,store_name,score,ward,review_cnt,review
0,0,1,らぁ麺 すぎ本,4.09,東京都内,413,TRY ラーメン大賞 2018-2019（講談社） 名店しょう油部門 第5位のお店であり、師...
1,1,1,らぁ麺 すぎ本,4.09,東京都内,413,西武新宿線の鷺ノ宮駅からすぐの所にある、ビブグルマン三年連続獲得のラーメン店【らぁ麺 すぎ本...
2,2,1,らぁ麺 すぎ本,4.09,東京都内,413,本日の昼はラーメン娘とこちらの店を初訪問。13時頃に到着すると、店の前には先客が5人待ちの状...
3,3,1,らぁ麺 すぎ本,4.09,東京都内,413,ゴールデンウィークで暇なので、都内の食べログ百名店を昨日に続いて訪れた。今日は、鷺宮にあるす...
4,4,1,らぁ麺 すぎ本,4.09,東京都内,413,新宿からの帰り道、どうしても尋常じゃないくらい美味いラーメンを食べたくなり鷺ノ宮駅で下車。新...
...,...,...,...,...,...,...,...
1151,1151,59,中華そば屋 伊藤,3.81,東京都内,483,中華そば屋伊藤。食べ手を選ぶ強烈な煮干しラーメンを出す店として知られています。煮干しラーメン...
1152,1152,59,中華そば屋 伊藤,3.81,東京都内,483,★4.2【FILE 1130】2017/03/05 (日) 14:04 訪問豊島の 南北線の...
1153,1153,59,中華そば屋 伊藤,3.81,東京都内,483,東京煮干ラーメンの聖地ということで、色々リセットするために何度かお邪魔しています。この日は正...
1154,1154,59,中華そば屋 伊藤,3.81,東京都内,483,王子神谷駅の近くに用事があったもので、そこから食べログで探してみるとなんと評価3.9のお店が...


https://qiita.com/toshiyuki_tsutsui/items/19590b464f15f845efcd

In [0]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.5)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.3)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already install

In [0]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :     tar => ok
[install-mecab-ipadic-NEologd] :     unxz => ok
[install-mecab-ipadic-NEologd] :     xargs => ok
[install-mecab-ipadic-NEologd] :     grep => ok
[install-mecab-ipadic-NEo

In [0]:
import numpy as np
import pandas as pd
import pickle
from gensim.models import word2vec
import MeCab

In [0]:
import MeCab
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
tagger=MeCab.Tagger("-d {0}".format(path))#タグはMeCab.Tagger（neologd辞書）を使用

In [0]:
tagger.parse('')
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]



#学習データの読み込み

df = pd.read_csv('tokyo_ramen_review.csv')
df_ramen = df.groupby(['store_name','score','review_cnt'])['review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)

#コーパス作成
wakati_ramen_text = []
for i in df_ramen['review']:
    txt = tokenize(i, 2, 10000, True)
    wakati_ramen_text.append(txt)
#↓↓↓↓↓↓↓ あなたの記事の内容
np.savetxt("ramen_corpus.txt", wakati_ramen_text,fmt='%s', delimiter=',')
#───────
np.savetxt("ramen_corpus.txt", wakati_ramen_text, fmt = '%s', delimiter = ',')
#↑↑↑↑↑↑↑ 編集リクエストの内容
# モデル作成
#↓↓↓↓↓↓↓ あなたの記事の内容
word2vec_ramen_model = word2vec.Word2Vec(wakati_ramen_text,sg=1,size=100, window=5,min_count=5,iter=100,workers=3)
#───────
word2vec_ramen_model = word2vec.Word2Vec(wakati_ramen_text, sg = 1, size = 100, window = 5, min_count = 5, iter = 100, workers = 3)
#↑↑↑↑↑↑↑ 編集リクエストの内容
#sg（0: CBOW, 1: skip-gram）,size（ベクトルの次元数）,window（学習に使う前後の単語数）,min_count（n回未満登場する単語を破棄）,iter（トレーニング反復回数）

# モデルのセーブ
word2vec_ramen_model.save("word2vec_ramen_model.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# モデルのロード
word2vec_ramen_model =word2vec.Word2Vec.load("word2vec_ramen_model.model")
word2vec_ramen_model.most_similar("スープ")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('旨味', 0.7881479859352112),
 ('風味', 0.6757103800773621),
 ('出汁', 0.6654976606369019),
 ('美味しい', 0.6606363654136658),
 ('香り', 0.634648323059082),
 ('コク', 0.6338876485824585),
 ('味わい', 0.6129070520401001),
 ('チャーシュー', 0.5965787172317505),
 ('これ', 0.5957608222961426),
 ('醤油', 0.5778023600578308)]

In [0]:
word2vec_ramen_model.most_similar(positive=[u"ラーメン"], negative=[u"スープ"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('2018', 0.3276529908180237),
 ('東京ラーメン', 0.326576292514801),
 ('最終', 0.3134579658508301),
 ('ランチ', 0.3113000690937042),
 ('ラーメン屋', 0.307037889957428),
 ('学生', 0.3069101870059967),
 ('019', 0.30417710542678833),
 ('定期的', 0.3021259903907776),
 ('獲得', 0.29894423484802246),
 ('不便', 0.29613471031188965)]

https://qiita.com/toshiyuki_tsutsui/items/b3ac8fd1b300c3404508

In [0]:
from collections import defaultdict
from gensim.models.keyedvectors import KeyedVectors
from sklearn.cluster import KMeans

model = KeyedVectors.load('word2vec_ramen_model.model')

max_vocab = 30000 #40000にしても結果は同じだった
vocab = list(model.wv.vocab.keys())[:max_vocab]
vectors = [model.wv[word] for word in vocab]

n_clusters = 6 #クラスター数はこちらで任意の値を定める
kmeans_model = KMeans(n_clusters=n_clusters, verbose=0, random_state=42, n_jobs=-1)
kmeans_model.fit(vectors)

cluster_labels = kmeans_model.labels_
cluster_to_words = defaultdict(list)
for cluster_id, word in zip(cluster_labels, vocab):
    cluster_to_words[cluster_id].append(word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
for words in cluster_to_words.values():
    print(words[:20])

['try', '大賞', '2018', '-2', '019', '名店', '部門', '師匠', '佐野実', 'リスペクト', '西武新宿線', 'ビブグルマン', '獲得', 'ラーメン店', 'すぎ', '支那そばや', '弟子', 'ぁめん', 'ミシュラン', '掲載']
['ラーメン', 'さん', 'なかっ', '今回', 'よく', '多い', '割合', 'わけ', 'それ', '好き', '美味しい', 'よう', 'ない', 'あと', 'どれ', 'ハイレベル', 'ここ', 'これ', '嬉しい', 'いい']
['機会', 'エリア', '平日', '開店', '時間', '11', '30', '先客', '11時', '10分', '20分', '到着', '30分', '20人', '訪問', '13時', '待ち', '状況', '我々', '15分']
['5人', '匂い', '店主', '入り口', '食券', 'カウンター', '５分', '入店', '店員', '5分', '丁寧', '綺麗', '出て', '最中', '製麺', 'スペース', '一部', '客席', '二人', '掛け']
['醤油', '1000円', '名古屋コーチン', '味付', '玉子', '100円', '大盛', '150円', '色々', 'ラインナップ', '醤油ラーメン', '券売機', '特製', '購入', 'ワンタン', '塩ラーメン', '本日', '二枚', '登場', '注文']
['チャーシュー', 'スープ', '多め', '食材', 'バランス', '融合', 'ダレ', 'タイプ', '中太', 'ストレート麺', '加水', 'シルキー', '食感', '相性', 'メンマ', '青ねぎ', 'ネギ', '九条ネギ', '抜群', '低温']


In [0]:
def change_dict_key(d, old_key, new_key, default_value=None):
    d[new_key] = d.pop(old_key, default_value)
change_dict_key(cluster_to_words, 0, '日付、お店の評価、ネット用語に関するワード')
change_dict_key(cluster_to_words, 1, '人や接客、内装に関するワード')
change_dict_key(cluster_to_words, 2, 'その他のワード')
change_dict_key(cluster_to_words, 3,  '券売機や注文に関するワード')
change_dict_key(cluster_to_words, 4, '曜日時間、店舗の地理的なワード')
change_dict_key(cluster_to_words, 5, 'ラーメンの中身に関するワード')

df_dict = pd.DataFrame.from_dict(cluster_to_words, orient="index").T
df_dict.iloc[:,[5,3,1,4,0,2]]

,ラーメンの中身に関するワード,券売機や注文に関するワード,人や接客、内装に関するワード,曜日時間、店舗の地理的なワード,日付、お店の評価、ネット用語に関するワード,その他のワード
0,機会,5人,try,醤油,チャーシュー,ラーメン
1,エリア,匂い,大賞,1000円,スープ,さん
2,平日,店主,2018,名古屋コーチン,多め,なかっ
3,開店,入り口,-2,味付,食材,今回
4,時間,食券,019,玉子,バランス,よく
...,...,...,...,...,...,...
837,None,None,None,None,ぬちまーす,None
838,None,None,None,None,朝日,None
839,None,None,None,None,ぶた,None
840,None,None,None,None,カラスミ,None


In [0]:
# 参考 https://qiita.com/tatsuya-miyamoto/items/f1539d86ad4980624111

from gensim import corpora
from gensim import models

taste_words = cluster_to_words['ラーメンの中身に関するワード']
kenbaiki_words = cluster_to_words['券売機や注文に関するワード']
taste_words.extend(kenbaiki_words)
ramen_word = taste_words
cluster_to_words.keys()

# 文書

f = open('ramen_corpus.txt','r',encoding="utf-8")
trainings = []

for i,data in enumerate(f):
    word = data.replace("'",'').replace('[','').replace(']','').replace(' ','').replace('\n','').split(",")
    trainings.append([i for i in word if i in ramen_word])

# 単語->id変換の辞書作成
dictionary = corpora.Dictionary(trainings)

# textsをcorpus化
corpus = list(map(dictionary.doc2bow,trainings))

# tfidf modelの生成
test_model = models.TfidfModel(corpus)

# corpusへのモデル適用
corpus_tfidf = test_model[corpus]

# id->単語へ変換
texts_tfidf = [] # id -> 単語表示に変えた文書ごとのTF-IDF
for doc in corpus_tfidf:
    text_tfidf = []
    for word in doc:
        text_tfidf.append([dictionary[word[0]],word[1]])
    texts_tfidf.append(text_tfidf)

from operator import itemgetter

texts_tfidf_sorted_top20 = [] 

#TF-IDF値を高い順に並び替え上位単語20個に絞る。
for i in range(len(texts_tfidf)):
    soted = sorted(texts_tfidf[i], key=itemgetter(1),reverse=True)
    soted_top20 = soted[:20]
    word_list = []
    for k in range(len(soted_top20)):
        word = soted_top20[k][0]
        word_list.append(word)
    texts_tfidf_sorted_top20.append(word_list)
# 結果をデータフレームに追加

df = pd.read_csv('tokyo_ramen_review.csv')
df_ramen = df.groupby(['store_name','score','review_cnt'])['review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)
df_ramen['texts_tfidf_sorted_top20'] = texts_tfidf_sorted_top20
df_ramen['id'] = ['ID-' + str(i + 1).zfill(6) for i in range(len(df_ramen.index))]
df_ramen_texts_tfidf_sorted_top20 = df_ramen.iloc[:,[5,0,1,2,4]].reset_index(drop=True)
df_ramen_texts_tfidf_sorted_top20
pickle.dump(df_ramen_texts_tfidf_sorted_top20, open('df_ramen_texts_tfidf_sorted_top20', 'wb'))

In [0]:
df_ramen_texts_tfidf_sorted_top20

,id,store_name,score,review_cnt,texts_tfidf_sorted_top20
0,ID-000001,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り..."
1,ID-000002,手打式超多加水麺 ののくら,4.02,300,"[亀有, 両さん, 縁起, 私たち, 回収, 南口, テボ, まり, やり方, タイル, 昼..."
2,ID-000003,中華そば しば田,3.99,668,"[仙川駅, 京王線, 整理券, タマ, 配布, 8時, push, 10時, 会社, 気温,..."
3,ID-000004,麺尊 RAGE,3.98,526,"[西荻窪, 西荻窪駅, ガレージ, ロック, ケロリン, 日本酒, 地域, 木箱, 世界観,..."
4,ID-000005,麺庵ちとせ,3.97,328,"[若松河田, ワンオペ, ボク, １０分, 住宅街, 連休, 2時, 奥様, 真面目, テン..."
5,ID-000006,メンドコロ キナリ,3.97,183,"[メンドコロ, キナリ, 東中野駅, テーブル席, 東口, ポールポジション, 伺い, ｊｒ..."
6,ID-000007,らーめん改,3.96,417,"[蔵前, 蔵前駅, ロック, 開放, 匂い, テボ, ３人, １０分, 二人, 日曜, 臨時..."
7,ID-000008,中華そば 多賀野,3.96,736,"[多賀野, 東急池上線, 荏原中延駅, 常連, 候補, 土日, 友人, チン, 1時間半, ..."
8,ID-000009,ほっこり中華そば もつけ,3.95,280,"[八王子駅, ナチュラル, 南口, ケロリン, どおり, 左折, 課題, 暑い, 内装, 文..."
9,ID-000010,麺屋 さくら井,3.95,324,"[さくら井, 三鷹駅, 三鷹, 北口, 12分, 住宅地, 書き, お盆, 休み, 住宅街,..."


In [0]:
from itertools import product

f = open('df_ramen_texts_tfidf_sorted_top20','rb')
store_df = pickle.load(f)
store_cross = []
for ids in product(store_df['id'], repeat=2):
    store_cross.append(ids)

store_cross_df = pd.DataFrame(store_cross, columns=['id_x', 'id_y'])

store_cross_detail = store_cross_df.merge(
    store_df[['id','store_name','score','review_cnt','texts_tfidf_sorted_top20']], how='inner', left_on='id_x', right_on='id'
).drop(columns='id').merge(
    store_df[['id','store_name','score','review_cnt','texts_tfidf_sorted_top20']], how='inner', left_on='id_y', right_on='id'
).drop(columns='id')
store_cross_detail = store_cross_detail[store_cross_detail['id_x'].isin(store_df['id'].loc[0:50])]
store_cross_detail = store_cross_detail.reset_index(drop=True).sort_values(['id_x'])

In [0]:
store_cross_detail

,id_x,id_y,store_name_x,score_x,review_cnt_x,texts_tfidf_sorted_top20_x,store_name_y,score_y,review_cnt_y,texts_tfidf_sorted_top20_y
0,ID-000001,ID-000001,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り..."
1020,ID-000001,ID-000021,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",らーめん飛粋,3.91,289,"[蒲田駅, 東口, 電柱, さま, ｊｒ, コップ, おしゃれ, 誘導, イケメン, jr,..."
1989,ID-000001,ID-000040,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",超純水採麺 天国屋,3.85,215,"[天国, 子ども, アイス, パチンコ, 超純水, 真面目, バス, 子供, 帰り道, 町田..."
408,ID-000001,ID-000009,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",ほっこり中華そば もつけ,3.95,280,"[八王子駅, ナチュラル, 南口, ケロリン, どおり, 左折, 課題, 暑い, 内装, 文..."
2601,ID-000001,ID-000052,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",真鯛らーめん 麺魚 本店,3.83,538,"[錦糸町駅, 公園, 10時, 熱い, 予想, 暑い, 江東橋, 南口, 土日, 20時, ..."
...,...,...,...,...,...,...,...,...,...,...
713,ID-000051,ID-000014,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら...",SOBA HOUSE 金色不如帰 新宿御苑本店,3.93,457,"[整理券, 新宿, タマ, テーブル席, 外国, 外国人, 20人, 土日, 18, 配布,..."
2294,ID-000051,ID-000045,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら...",中華そば 和渦 TOKYO,3.84,173,"[北品川駅, 品川駅, 歩道橋, テーブル席, 京急, ビル, ２人, 山手線, カフェ, ..."
1835,ID-000051,ID-000036,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら...",麺屋一燈,3.86,1438,"[整理券, 予約, epark, 新小岩駅, 指定, システム, 配布, 男子, 受付, 整..."
815,ID-000051,ID-000016,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら...",純手打ち だるま,3.92,159,"[中野富士見町駅, だるま, 中野富士見町, 気配り, 右折, 主人, プラン, 湯切り, ..."


In [0]:
##ラーメン店xに対してラーメン店yの類似度を算出

import itertools
from tqdm import tqdm 

#コサイン類似度を算出する関数を定義
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

#cossimだけの組み合わせ（同じワード同士の組みあわせがでてくるため）
#2次元を１次元にする　setが重複を削除てきなやつ。
uniq_words = list(set(itertools.chain.from_iterable(store_df['texts_tfidf_sorted_top20'].values)))
scores = {}
for word1, word2 in product(uniq_words, repeat=2):
    scores[(word1, word2)] =  cos_sim(word2vec_ramen_model.wv[word1],word2vec_ramen_model.wv[word2]) 

In [0]:
avg_avg_scores = []
for i in tqdm(range(len(store_cross_detail['texts_tfidf_sorted_top20_x']))):
    avg_scores = []
    for j in range(len(store_cross_detail['texts_tfidf_sorted_top20_x'][i])):
        word_cross_scores = []
        word_a = store_cross_detail['texts_tfidf_sorted_top20_x'][i][j]
        for k in range(len(store_cross_detail['texts_tfidf_sorted_top20_y'][i])):
            word_b = store_cross_detail['texts_tfidf_sorted_top20_y'][i][k]
            score = scores[(word_a, word_b)]#単語間のスコアを出す。
            word_cross_scores.append(score)
        avg_scores.append(np.mean(word_cross_scores))#20個の単語間スコアの平均値
    avg_avg_scores.append(np.mean(avg_scores))#20個の単語間スコアの平均値の平均値

100%|██████████| 3009/3009 [00:32<00:00, 92.67it/s]


In [0]:
store_cross_detail

,id_x,id_y,store_name_x,score_x,review_cnt_x,texts_tfidf_sorted_top20_x,store_name_y,score_y,review_cnt_y,texts_tfidf_sorted_top20_y
0,ID-000001,ID-000001,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り..."
1020,ID-000001,ID-000021,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",らーめん飛粋,3.91,289,"[蒲田駅, 東口, 電柱, さま, ｊｒ, コップ, おしゃれ, 誘導, イケメン, jr,..."
1989,ID-000001,ID-000040,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",超純水採麺 天国屋,3.85,215,"[天国, 子ども, アイス, パチンコ, 超純水, 真面目, バス, 子供, 帰り道, 町田..."
408,ID-000001,ID-000009,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",ほっこり中華そば もつけ,3.95,280,"[八王子駅, ナチュラル, 南口, ケロリン, どおり, 左折, 課題, 暑い, 内装, 文..."
2601,ID-000001,ID-000052,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",真鯛らーめん 麺魚 本店,3.83,538,"[錦糸町駅, 公園, 10時, 熱い, 予想, 暑い, 江東橋, 南口, 土日, 20時, ..."
...,...,...,...,...,...,...,...,...,...,...
713,ID-000051,ID-000014,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら...",SOBA HOUSE 金色不如帰 新宿御苑本店,3.93,457,"[整理券, 新宿, タマ, テーブル席, 外国, 外国人, 20人, 土日, 18, 配布,..."
2294,ID-000051,ID-000045,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら...",中華そば 和渦 TOKYO,3.84,173,"[北品川駅, 品川駅, 歩道橋, テーブル席, 京急, ビル, ２人, 山手線, カフェ, ..."
1835,ID-000051,ID-000036,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら...",麺屋一燈,3.86,1438,"[整理券, 予約, epark, 新小岩駅, 指定, システム, 配布, 男子, 受付, 整..."
815,ID-000051,ID-000016,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら...",純手打ち だるま,3.92,159,"[中野富士見町駅, だるま, 中野富士見町, 気配り, 右折, 主人, プラン, 湯切り, ..."


In [0]:
store_cross_detail.insert(6, 'avg_cos_sim_rate', avg_avg_scores)  

## 「二郎」と類似度が高いラーメン屋を高い順に表示


In [0]:
# 「二郎」と類似度が高いラーメン屋を高い順に表示
store_cross_detail = store_cross_detail.sort_values(['id_x', 'avg_cos_sim_rate'], ascending=[True, False])
df_sim_x = store_cross_detail[store_cross_detail['store_name_x'].str.contains('ラーメン')]
df_sim_x.reset_index(drop=True)
df_sim_x

,id_x,id_y,store_name_x,score_x,review_cnt_x,texts_tfidf_sorted_top20_x,avg_cos_sim_rate,store_name_y,score_y,review_cnt_y,texts_tfidf_sorted_top20_y
25,ID-000026,ID-000001,ラーメン屋 トイ・ボックス,3.89,583,"[ボックス, 三ノ輪, せし, 日比谷線, 待た, ロット, 1本, 回収, 候補, 猛暑,...",0.263029,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り..."
2218,ID-000026,ID-000044,ラーメン屋 トイ・ボックス,3.89,583,"[ボックス, 三ノ輪, せし, 日比谷線, 待た, ロット, 1本, 回収, 候補, 猛暑,...",0.254296,中華そば 満鶏軒,3.85,352,"[錦糸町駅, 江東橋, 公園, 南口, 向かい, 二丁目, 主人, 左折, ブロック, ロー..."
280,ID-000026,ID-000006,ラーメン屋 トイ・ボックス,3.89,583,"[ボックス, 三ノ輪, せし, 日比谷線, 待た, ロット, 1本, 回収, 候補, 猛暑,...",0.242056,メンドコロ キナリ,3.97,183,"[メンドコロ, キナリ, 東中野駅, テーブル席, 東口, ポールポジション, 伺い, ｊｒ..."
535,ID-000026,ID-000011,ラーメン屋 トイ・ボックス,3.89,583,"[ボックス, 三ノ輪, せし, 日比谷線, 待た, ロット, 1本, 回収, 候補, 猛暑,...",0.237164,RAMEN にじゅうぶんのいち,3.94,201,"[東尾久, 都電, 照明, 三ノ輪, 基調, 木目, 待合, 木箱, 商店街, 背後, シッ..."
2116,ID-000026,ID-000042,ラーメン屋 トイ・ボックス,3.89,583,"[ボックス, 三ノ輪, せし, 日比谷線, 待た, ロット, 1本, 回収, 候補, 猛暑,...",0.233351,町田汁場 しおらーめん進化 本店,3.85,514,"[天国, 駅前, 町田街道, 音楽, 駐車場, 文字, 遠い, 小田急, バス, 予約, ひ..."
...,...,...,...,...,...,...,...,...,...,...,...
1819,ID-000035,ID-000036,ラーメン巌哲,3.87,292,"[日本酒, 秋刀魚, 早稲田駅, 都電, 日傘, ポスター, 常連, 我が家, 作成, 書き...",0.178977,麺屋一燈,3.86,1438,"[整理券, 予約, epark, 新小岩駅, 指定, システム, 配布, 男子, 受付, 整..."
595,ID-000035,ID-000012,ラーメン巌哲,3.87,292,"[日本酒, 秋刀魚, 早稲田駅, 都電, 日傘, ポスター, 常連, 我が家, 作成, 書き...",0.178181,中華蕎麦にし乃,3.94,379,"[乃木坂46, おしぼり, お酢, さま, 地下鉄, ウェイティング, 丸ノ内線, 午前中,..."
2431,ID-000035,ID-000048,ラーメン巌哲,3.87,292,"[日本酒, 秋刀魚, 早稲田駅, 都電, 日傘, ポスター, 常連, 我が家, 作成, 書き...",0.176801,スープメン,3.83,49,"[おしぼり, お客, デカ, ウェイティング, open, ドア, 常備, 経営, 55, ..."
2227,ID-000035,ID-000044,ラーメン巌哲,3.87,292,"[日本酒, 秋刀魚, 早稲田駅, 都電, 日傘, ポスター, 常連, 我が家, 作成, 書き...",0.175346,中華そば 満鶏軒,3.85,352,"[錦糸町駅, 江東橋, 公園, 南口, 向かい, 二丁目, 主人, 左折, ブロック, ロー..."


In [0]:
def min_max(x, axis=None):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)
    return result

In [0]:
b = df_sim_x['avg_cos_sim_rate']
b

25      0.263029
2218    0.254296
280     0.242056
535     0.237164
2116    0.233351
          ...   
1819    0.178977
595     0.178181
2431    0.176801
2227    0.175346
1054    0.171688
Name: avg_cos_sim_rate, Length: 118, dtype: float64

In [0]:
c = min_max(b.values)
c

array([0.8498417 , 0.76859302, 0.65471349, 0.60920025, 0.57371922,
       0.56627267, 0.54746656, 0.51353165, 0.50480239, 0.49245779,
       0.48221498, 0.47975909, 0.46455834, 0.45217755, 0.43427108,
       0.42312572, 0.41322841, 0.40194885, 0.39641901, 0.39548402,
       0.38625509, 0.38574046, 0.38306954, 0.37895522, 0.37654134,
       0.37396969, 0.36898818, 0.36768675, 0.35738877, 0.35710012,
       0.35254686, 0.34655854, 0.34600508, 0.32233156, 0.31315227,
       0.2841053 , 0.28155735, 0.27726752, 0.27347222, 0.26915687,
       0.26712273, 0.25963639, 0.25863942, 0.25814114, 0.25623608,
       0.25621778, 0.23492888, 0.23154396, 0.21579641, 0.20340064,
       0.20168427, 0.19924918, 0.1869323 , 0.18672656, 0.18102882,
       0.1653542 , 0.16446745, 0.1592386 , 0.04591988, 1.        ,
       0.53858798, 0.47744919, 0.44819578, 0.44809707, 0.44336566,
       0.42191982, 0.40518501, 0.40423809, 0.37669482, 0.37067655,
       0.35063528, 0.34922128, 0.34540033, 0.34137156, 0.32589

In [0]:
df_sim_x.insert(7, '正規化', c)
df_sim_x

,id_x,id_y,store_name_x,score_x,review_cnt_x,texts_tfidf_sorted_top20_x,avg_cos_sim_rate,正規化,store_name_y,score_y,review_cnt_y,texts_tfidf_sorted_top20_y
25,ID-000026,ID-000001,ラーメン屋 トイ・ボックス,3.89,583,"[ボックス, 三ノ輪, せし, 日比谷線, 待た, ロット, 1本, 回収, 候補, 猛暑,...",0.263029,0.849842,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り..."
2218,ID-000026,ID-000044,ラーメン屋 トイ・ボックス,3.89,583,"[ボックス, 三ノ輪, せし, 日比谷線, 待た, ロット, 1本, 回収, 候補, 猛暑,...",0.254296,0.768593,中華そば 満鶏軒,3.85,352,"[錦糸町駅, 江東橋, 公園, 南口, 向かい, 二丁目, 主人, 左折, ブロック, ロー..."
280,ID-000026,ID-000006,ラーメン屋 トイ・ボックス,3.89,583,"[ボックス, 三ノ輪, せし, 日比谷線, 待た, ロット, 1本, 回収, 候補, 猛暑,...",0.242056,0.654713,メンドコロ キナリ,3.97,183,"[メンドコロ, キナリ, 東中野駅, テーブル席, 東口, ポールポジション, 伺い, ｊｒ..."
535,ID-000026,ID-000011,ラーメン屋 トイ・ボックス,3.89,583,"[ボックス, 三ノ輪, せし, 日比谷線, 待た, ロット, 1本, 回収, 候補, 猛暑,...",0.237164,0.609200,RAMEN にじゅうぶんのいち,3.94,201,"[東尾久, 都電, 照明, 三ノ輪, 基調, 木目, 待合, 木箱, 商店街, 背後, シッ..."
2116,ID-000026,ID-000042,ラーメン屋 トイ・ボックス,3.89,583,"[ボックス, 三ノ輪, せし, 日比谷線, 待た, ロット, 1本, 回収, 候補, 猛暑,...",0.233351,0.573719,町田汁場 しおらーめん進化 本店,3.85,514,"[天国, 駅前, 町田街道, 音楽, 駐車場, 文字, 遠い, 小田急, バス, 予約, ひ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1819,ID-000035,ID-000036,ラーメン巌哲,3.87,292,"[日本酒, 秋刀魚, 早稲田駅, 都電, 日傘, ポスター, 常連, 我が家, 作成, 書き...",0.178977,0.067821,麺屋一燈,3.86,1438,"[整理券, 予約, epark, 新小岩駅, 指定, システム, 配布, 男子, 受付, 整..."
595,ID-000035,ID-000012,ラーメン巌哲,3.87,292,"[日本酒, 秋刀魚, 早稲田駅, 都電, 日傘, ポスター, 常連, 我が家, 作成, 書き...",0.178181,0.060413,中華蕎麦にし乃,3.94,379,"[乃木坂46, おしぼり, お酢, さま, 地下鉄, ウェイティング, 丸ノ内線, 午前中,..."
2431,ID-000035,ID-000048,ラーメン巌哲,3.87,292,"[日本酒, 秋刀魚, 早稲田駅, 都電, 日傘, ポスター, 常連, 我が家, 作成, 書き...",0.176801,0.047577,スープメン,3.83,49,"[おしぼり, お客, デカ, ウェイティング, open, ドア, 常備, 経営, 55, ..."
2227,ID-000035,ID-000044,ラーメン巌哲,3.87,292,"[日本酒, 秋刀魚, 早稲田駅, 都電, 日傘, ポスター, 常連, 我が家, 作成, 書き...",0.175346,0.034035,中華そば 満鶏軒,3.85,352,"[錦糸町駅, 江東橋, 公園, 南口, 向かい, 二丁目, 主人, 左折, ブロック, ロー..."


In [0]:
store_cross_detail = store_cross_detail.sort_values(['id_x', 'avg_cos_sim_rate'], ascending=[True, False])
store_cross_detail

,id_x,id_y,store_name_x,score_x,review_cnt_x,texts_tfidf_sorted_top20_x,avg_cos_sim_rate,store_name_y,score_y,review_cnt_y,texts_tfidf_sorted_top20_y
0,ID-000001,ID-000001,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",0.259489,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り..."
1785,ID-000001,ID-000036,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",0.259382,麺屋一燈,3.86,1438,"[整理券, 予約, epark, 新小岩駅, 指定, システム, 配布, 男子, 受付, 整..."
969,ID-000001,ID-000020,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",0.228441,Homemade Ramen 麦苗,3.91,639,"[e3, ウェイティング, 大森駅, 受付, 実家, 私たち, 日傘, 50人, 方式, 人..."
1632,ID-000001,ID-000033,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",0.223252,RAMEN GOTTSU,3.87,296,"[練馬駅, 練馬, ごっつええ感じ, お子様, 周辺, ブラウン, お洒落, お客様, 待合..."
2550,ID-000001,ID-000051,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り...",0.221671,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら..."
...,...,...,...,...,...,...,...,...,...,...,...
2906,ID-000051,ID-000057,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら...",0.175797,中華そば屋 伊藤,3.81,483,"[王子神谷駅, 親父, 王子駅, お金, 嬉しく, ひとつ, 商店街, 営業中, 最寄り, ..."
50,ID-000051,ID-000001,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら...",0.172219,らぁ麺 すぎ本,4.09,413,"[傘立て, 褒美, なのだ, 便利, 女の子, 製麺機, ナチュラル, 最中, 下車, 帰り..."
1580,ID-000051,ID-000031,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら...",0.171530,らぁ麺やまぐち,3.88,269,"[早稲田通り, 早稲田駅, 高田馬場駅, ベンチ, トレー, テーブル席, バイト, 雑誌,..."
2345,ID-000051,ID-000046,麺 みつヰ,3.83,355,"[みつ, 浅草, 女将, 路地, マスター, きり, 直前, ピカ, 前払い, お金, あら...",0.167690,らぁめん小池,3.84,370,"[上北沢駅, 村上, 水原, 京王線, ランチタイム, 意識, 退店, 14, 北口, 暑い..."


In [0]:
df_sim_x = store_cross_detail[store_cross_detail['store_name_x'].str.contains('くろ喜')]
df_sim_x

,id_x,id_y,store_name_x,score_x,review_cnt_x,texts_tfidf_sorted_top20_x,avg_cos_sim_rate,store_name_y,score_y,review_cnt_y,texts_tfidf_sorted_top20_y
2921,ID-000015,ID-000058,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.323794,煮干しつけ麺 宮元,3.81,549,"[蒲田駅, チン, ブロック, １４時, マスター, おじさん, ポット, 整理, 定刻, ..."
473,ID-000015,ID-000010,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.254533,麺屋 さくら井,3.95,324,"[さくら井, 三鷹駅, 三鷹, 北口, 12分, 住宅地, 書き, お盆, 休み, 住宅街,..."
2360,ID-000015,ID-000047,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.246968,中華そば 堀川,3.83,47,"[お子様, 二人, 移動, フリーク, 多数, 住宅街, 居心地, 清潔, 15時, 指示,..."
167,ID-000015,ID-000004,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.223652,麺尊 RAGE,3.98,526,"[西荻窪, 西荻窪駅, ガレージ, ロック, ケロリン, 日本酒, 地域, 木箱, 世界観,..."
218,ID-000015,ID-000005,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.221406,麺庵ちとせ,3.97,328,"[若松河田, ワンオペ, ボク, １０分, 住宅街, 連休, 2時, 奥様, 真面目, テン..."
1901,ID-000015,ID-000038,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.217025,中華そば 児ノ木,3.86,164,"[落合駅, 東京メトロ東西線, 製麺, テーブル席, ピカ, １４時, 決意, open, ..."
1340,ID-000015,ID-000027,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.214921,寿製麺 よしかわ 西台駅前店,3.89,150,"[西台, 西台駅, 駅前, 都営三田線, 待ち行列, 狭く, カップル, 時間的, 職人, ..."
1952,ID-000015,ID-000039,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.211945,麺や金時,3.85,228,"[江古田駅, ワンタン麺, お仕事, ボックス, 定刻, 大将, 沿線, 訪店, 住宅街, ..."
2105,ID-000015,ID-000042,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.211282,町田汁場 しおらーめん進化 本店,3.85,514,"[天国, 駅前, 町田街道, 音楽, 駐車場, 文字, 遠い, 小田急, バス, 予約, ひ..."
2207,ID-000015,ID-000044,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.210799,中華そば 満鶏軒,3.85,352,"[錦糸町駅, 江東橋, 公園, 南口, 向かい, 二丁目, 主人, 左折, ブロック, ロー..."


In [0]:
df_sim_x.reset_index(drop=True)
df_sim_x

,id_x,id_y,store_name_x,score_x,review_cnt_x,texts_tfidf_sorted_top20_x,avg_cos_sim_rate,store_name_y,score_y,review_cnt_y,texts_tfidf_sorted_top20_y
2921,ID-000015,ID-000058,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.323794,煮干しつけ麺 宮元,3.81,549,"[蒲田駅, チン, ブロック, １４時, マスター, おじさん, ポット, 整理, 定刻, ..."
473,ID-000015,ID-000010,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.254533,麺屋 さくら井,3.95,324,"[さくら井, 三鷹駅, 三鷹, 北口, 12分, 住宅地, 書き, お盆, 休み, 住宅街,..."
2360,ID-000015,ID-000047,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.246968,中華そば 堀川,3.83,47,"[お子様, 二人, 移動, フリーク, 多数, 住宅街, 居心地, 清潔, 15時, 指示,..."
167,ID-000015,ID-000004,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.223652,麺尊 RAGE,3.98,526,"[西荻窪, 西荻窪駅, ガレージ, ロック, ケロリン, 日本酒, 地域, 木箱, 世界観,..."
218,ID-000015,ID-000005,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.221406,麺庵ちとせ,3.97,328,"[若松河田, ワンオペ, ボク, １０分, 住宅街, 連休, 2時, 奥様, 真面目, テン..."
1901,ID-000015,ID-000038,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.217025,中華そば 児ノ木,3.86,164,"[落合駅, 東京メトロ東西線, 製麺, テーブル席, ピカ, １４時, 決意, open, ..."
1340,ID-000015,ID-000027,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.214921,寿製麺 よしかわ 西台駅前店,3.89,150,"[西台, 西台駅, 駅前, 都営三田線, 待ち行列, 狭く, カップル, 時間的, 職人, ..."
1952,ID-000015,ID-000039,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.211945,麺や金時,3.85,228,"[江古田駅, ワンタン麺, お仕事, ボックス, 定刻, 大将, 沿線, 訪店, 住宅街, ..."
2105,ID-000015,ID-000042,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.211282,町田汁場 しおらーめん進化 本店,3.85,514,"[天国, 駅前, 町田街道, 音楽, 駐車場, 文字, 遠い, 小田急, バス, 予約, ひ..."
2207,ID-000015,ID-000044,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.210799,中華そば 満鶏軒,3.85,352,"[錦糸町駅, 江東橋, 公園, 南口, 向かい, 二丁目, 主人, 左折, ブロック, ロー..."


In [0]:
def min_max(x, axis=None):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)#![x_y_隠れた名店度.jpg](https://qiita-image-store.s3.amazonaws.com/0/327405/66e75204-2d00-f399-d62e-f01ed492633e.jpeg)

    return result
b = df_sim_x['avg_cos_sim_rate']
c = min_max(b.values)
df_sim_x.insert(7, '正規化', c)

d = df_sim_x['review_cnt_y']
e = 1-min_max(d.values)
df_sim_x.insert(11, 'レビュー数_正規化', e)
f = df_sim_x['正規化']*(df_sim_x['レビュー数_正規化'])
df_sim_x.insert(9, '隠れた名店_score', f)
df_kakureta_meiten = df_sim_x.sort_values('隠れた名店_score', ascending=False)
df_kakureta_meiten[df_kakureta_meiten['review_cnt_y'] < 100]

,id_x,id_y,store_name_x,score_x,review_cnt_x,texts_tfidf_sorted_top20_x,avg_cos_sim_rate,正規化,store_name_y,隠れた名店_score,score_y,review_cnt_y,レビュー数_正規化,texts_tfidf_sorted_top20_y
2360,ID-000015,ID-000047,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.246968,0.533328,中華そば 堀川,0.533328,3.83,47,1.000000,"[お子様, 二人, 移動, フリーク, 多数, 住宅街, 居心地, 清潔, 15時, 指示,..."
1136,ID-000015,ID-000023,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.202493,0.263169,キング製麺,0.253331,3.90,99,0.962617,"[ワンタン麺, 王子駅, 水原, レトロ, 利用, 製麺, 人数, 午後2時, 都電, 歩道..."
2411,ID-000015,ID-000048,饗 くろ喜,3.93,893,"[秋葉原駅, もてなし, 胡椒, 大将, 用事, がけ, タマ, 検索, 常連, 夏休み, ...",0.195153,0.218582,スープメン,0.218268,3.83,49,0.998562,"[おしぼり, お客, デカ, ウェイティング, open, ドア, 常備, 経営, 55, ..."
